## Prepare and Merge Datasets for Visualiation

In [49]:
import pandas as pd
import numpy as np

# open df from csv
position_366096 = pd.read_csv('lqty_weth_366096.csv')
position_453456 = pd.read_csv('lqty_weth_453456.csv')
position_506262 = pd.read_csv('lqty_weth_506262.csv')
range = pd.read_csv('lqtyeth_range_position.csv')

# in range df if upper_price and lower_price are same, set both to NA
range.loc[range['upper_price'] == range['lower_price'], ['upper_price', 'lower_price']] = 0

range['daily_LVR_ratio_full'] = (range['log_return'] ** 2) / 8
range['daily_LVR_full_USD'] = range['daily_LVR_ratio_full'] * range['pool_value'] * range['token1_usd'] / 10000
range['cumulative_LVR_ratio_full'] = range['daily_LVR_ratio_full'].cumsum()
range['cumulative_LVR_full_USD'] = range['daily_LVR_full_USD'].cumsum()

# combine range, position_250274, position_515539, position_520645 on date
df = pd.merge(range, position_366096, on='date', how='left', suffixes=('', '_366096'))
df = pd.merge(df, position_453456, on='date', how='left', suffixes=('', '_453456'))
df = pd.merge(df, position_506262, on='date', how='left', suffixes=('', '_506262'))
combined_df = df


In [50]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import plotly.offline as pyo

# Convert 'date' column to datetime if it's not already
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Create cumulative columns
combined_df['cumulative_LVR_ratio'] = combined_df['daily_LVR_ratio'].cumsum()
combined_df['cumulative_LVR_366096'] = combined_df['daily_LVR_ratio_366096'].cumsum()
combined_df['cumulative_LVR_453456'] = combined_df['daily_LVR_ratio_453456'].cumsum()
combined_df['cumulative_LVR_506262'] = combined_df['daily_LVR_ratio_506262'].cumsum()

combined_df['daily_LVR_USD'] = combined_df['daily_LVR'] * combined_df['token1_usd']
combined_df['daily_LVR_USD_366096'] = combined_df['daily_LVR_366096'] * combined_df['token1_usd']
combined_df['daily_LVR_USD_453456'] = combined_df['daily_LVR_453456'] * combined_df['token1_usd']
combined_df['daily_LVR_USD_506262'] = combined_df['daily_LVR_506262'] * combined_df['token1_usd']

combined_df['cumulative_LVR_USD'] = combined_df['daily_LVR_USD'].cumsum()
combined_df['cumulative_LVR_USD_366096'] = combined_df['daily_LVR_USD_366096'].cumsum()
combined_df['cumulative_LVR_USD_453456'] = combined_df['daily_LVR_USD_453456'].cumsum()
combined_df['cumulative_LVR_USD_506262'] = combined_df['daily_LVR_USD_506262'].cumsum()

# Create interactive figure with subplots
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

# Add traces for cumulative LVR
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['cumulative_LVR_USD']/(combined_df['pool_value']*combined_df['token1_usd']),mode='lines', name='Range LQTY/ETH Active Strategy', line=dict(color='red', width=3)))
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['cumulative_LVR_full_USD']/(combined_df['pool_value']*combined_df['token1_usd']),mode='lines', name='Full Range / V2 Reference Position', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['cumulative_LVR_USD_366096']/(combined_df['pool_value_366096']*combined_df['token1_usd']),mode='lines', name='V3 Position 0.3% 366096', line=dict(color='green')))
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['cumulative_LVR_USD_453456']/(combined_df['pool_value_453456']*combined_df['token1_usd']),mode='lines', name='V3 Position 0.3% 453456', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['cumulative_LVR_USD_506262']/(combined_df['pool_value_506262']*combined_df['token1_usd']),mode='lines', name='V3 Position 0.3% 506262', line=dict(color='blue')))

# Add trace for log return on a separate y-axis
fig.add_trace(go.Scatter(x=combined_df['date'], y=combined_df['log_return'],mode='lines', name='Log Return', line=dict(color='black')),row=2, col=1)

# Add traces for current price, upper price, and lower price on a separate y-axis
fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['current_price'],mode='lines', name='Current Price', line=dict(color='black')),row=3, col=1)

fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['upper_price'],mode='lines', name='Range Vault Upper Price', line=dict(color='red')),row=3, col=1)
fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['lower_price'],mode='lines', name='Range Vault Lower Price', line=dict(color='red')),row=3, col=1)

fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['upper_price_366096'],mode='lines', name='V3 Position 0.3% 366096 Upper Price', line=dict(color='green')),row=3, col=1)
fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['lower_price_366096'],mode='lines', name='V3 Position 0.3% 366096 Lower Price', line=dict(color='green')),row=3, col=1)

fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['upper_price_453456'],mode='lines', name='V3 Position 0.3% 453456 Upper Price', line=dict(color='orange')),row=3, col=1)
fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['lower_price_453456'],mode='lines', name='V3 Position 0.3% 453456 Lower Price', line=dict(color='orange')),row=3, col=1)

fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['upper_price_506262'],mode='lines', name='V3 Position 0.3% 506262 Upper Price', line=dict(color='blue')),row=3, col=1)
fig.add_trace(go.Scatter(x=combined_df['date'], y=1/combined_df['lower_price_506262'],mode='lines', name='V3 Position 0.3% 506262 Lower Price', line=dict(color='blue')),row=3, col=1)

# Set layout options
fig.update_layout(
    title='Cumulative LVR as % of Pool Value for LQTYETH',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Cumulative LVR'),
    yaxis2=dict(title='Volatility %'),
    yaxis3=dict(title='Price'),
    legend=dict(x=1.1, y=1),
    autosize=True,
    margin=dict(l=50, r=50, t=80, b=50),
    hovermode='x unified'
)

# Save the chart as an HTML file
chart_filename = 'LQTYETH.html'
pyo.plot(fig, filename=chart_filename, auto_open=False)

# Display the interactive chart
fig.show()
